# Elastic Weld Analysis <v0.1>

This program is for the purpose of analysis of weld groups using the Elastic Method, as defined in the Steel Construction Manual.

## Assumptions and Limitations

1. This program only evaluates the strength of the weld metal. The strength of the base metal should also be considered.
2. The same weld size must be used for all weld lines since a unit throat size has been assumed in calculations.
3. The required weld size is calculated for a fillet weld.
4. AISC limitations on minimum and maximum weld sizes and lengths are not enforced. These limitations should be checked by the user.
5. The maximum load is determined from the vectorial combination of the stress at the element furthest from the CG of the weld group.

## Inputs:

In [1]:
from elastic_weld_analysis import Weld_Load, Weld, Weld_Group
import ipywidgets as wg
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
%matplotlib inline


In [2]:
class Weld_Layout(object):
    
    def __init__(self, model):
        
        self.model = model

        # Weld segments accordian.
        # Input widgets.
        self.segment_name = wg.Text(value='name me', description='name')
        self.xi = wg.FloatText(value=0.0, description=r'$x_i$ &nbsp <i>[in]</i>')
        self.xj = wg.FloatText(value=0.0, description=r'$y_i$ &nbsp <i>[in]</i>')
        self.yi = wg.FloatText(value=0.0, description=r'$x_j$ &nbsp <i>[in]</i>')
        self.yj = wg.FloatText(value=0.0, description=r'$y_j$ &nbsp <i>[in]</i>')        
        # Add segment button.
        self.add_segment_btn = wg.Button(description='Add/Update Segment')
        self.add_segment_btn.on_click(self.add_segment)
        # Delete segment button.
        self.del_segment_btn = wg.Button(description='Delete Segment')
        self.del_segment_btn.on_click(self.del_segment)
        # Weld segment selection box.
        opts = [str(v.name) + ": " + str((v.i[0], v.i[1])) + " " + str((v.j[0], v.j[1])) for k, v in self.model.welds.items()]
        self.weld_segments = wg.Select(description='Weld Segments', options=opts)
        self.weld_segments.observe(self.on_segment_change, names='value')
        # Welds accordian.
        self.welds_box = wg.VBox([self.segment_name, 
                                  self.xi, 
                                  self.xj, 
                                  self.yi, 
                                  self.yj, 
                                  self.add_segment_btn,
                                  self.del_segment_btn,
                                  self.weld_segments])
        self.welds_accordian = wg.Accordion(children=[self.welds_box])
        self.welds_accordian.set_title(0, 'Weld Segments')
        
        # Loads accordian.
        self.load_name = wg.Text(value='name me', description='name')
        self.pz = wg.FloatText(value=0.0, description=r'$P_z$ &nbsp <i>[k]</i>')
        self.vx = wg.FloatText(value=0.0, description=r'$V_x$ &nbsp <i>[k]</i>')
        self.vy = wg.FloatText(value=0.0, description=r'$V_y$ &nbsp <i>[k]</i>')
        self.mx = wg.FloatText(value=0.0, description=r'$M_x$ &nbsp <i>[k*in]</i>')
        self.my = wg.FloatText(value=0.0, description=r'$M_y$ &nbsp <i>[k*in]</i>')
        self.mz = wg.FloatText(value=0.0, description=r'$M_z$ &nbsp <i>[k*in]</i>')
        # Add load button.
        self.add_load_btn = wg.Button(description='Add/Update Load')
        self.add_load_btn.on_click(self.add_load)
        # Delete load button.
        self.del_load_btn = wg.Button(description='Delete Load')
        self.del_load_btn.on_click(self.del_load)
        # Load selection box.
        opts = [str(v.name) + ": " + str([l for l in v.load]) for k, v in self.model.loads.items()]
        self.loads_select = wg.Select(description='Loads', options=opts)
        self.loads_select.observe(self.on_load_change, names='value')
        # Accordian
        self.loads_box = wg.VBox([self.load_name, 
                                  self.pz, 
                                  self.vx, 
                                  self.vy, 
                                  self.mx,
                                  self.my,
                                  self.mz,
                                  self.add_load_btn,
                                  self.del_load_btn,
                                  self.loads_select])
        self.loads_accordian = wg.Accordion(children=[self.loads_box])
        self.loads_accordian.set_title(0, 'Loads')
        
        # Calc button.
        self.calc_btn = wg.Button(description='Calc')
        self.calc_btn.on_click(self.calc)
        
        # Output
        self.output = wg.Output()
        with self.output:
            print('needs calc')

        self.output_accordian = wg.Accordion(children=[self.output])
        self.output_accordian.set_title(0, 'Results')

        # Build layout.
        self.layout = wg.VBox([self.welds_accordian,
                                     self.loads_accordian,
                                     self.calc_btn, 
                                     self.output_accordian])
        display(self.layout)        

    def on_segment_change(self, change):
        key = change['new'][:change['new'].find(':')]
        self.segment_name.value = key
        self.xi.value = self.model.welds[key].i[0]
        self.xj.value = self.model.welds[key].i[1]
        self.yi.value = self.model.welds[key].j[0]
        self.yj.value = self.model.welds[key].j[1]
        
    def add_segment(self, b):
        # Update model.
        self.model.new_segment(Weld(name=self.segment_name.value, 
                                    xi=self.xi.value,
                                    yi=self.yi.value,
                                    xj=self.xj.value,
                                    yj=self.yj.value))
        # Update layout
        self.update_segments()

    def del_segment(self, b):
        # Update model.
        self.model.del_segment(self.segment_name.value)
        # Update layout.
        self.update_segments()

    def update_segments(self):
        # Update layout.
        opts = [str(v.name) + ": " + str((v.i[0], v.i[1])) + " " + str((v.j[0], v.j[1])) for k, v in self.model.welds.items()]
        self.weld_segments.options = opts

    def on_load_change(self, change):
        key = change['new'][:change['new'].find(':')]
        self.load_name.value = key
        self.pz.value = self.model.loads[key].load[0]
        self.vx.value = self.model.loads[key].load[1]
        self.vy.value = self.model.loads[key].load[2]
        self.mx.value = self.model.loads[key].load[3]
        self.my.value = self.model.loads[key].load[4]
        self.mz.value = self.model.loads[key].load[5]
        
    def add_load(self, b):
        # Update model.
        self.model.new_load(Weld_Load(name=self.load_name.value, 
                                      pz=self.pz.value,
                                      vx=self.vy.value,
                                      vy=self.vx.value,
                                      mx=self.mx.value, 
                                      my=self.my.value, 
                                      mz=self.mz.value))
        # Update layout.
        self.update_loads()

    def del_load(self, b):
        # Update model.
        self.model.del_load(self.load_name.value)
        # Update layout.
        self.update_loads()

    def update_loads(self):
        # Update layout.
        opts = [str(v.name) + ": " + str([l for l in v.load]) for k, v in self.model.loads.items()]
        self.loads_select.options = opts
        
    def calc(self, b):
        
        self.model.analyze()
        self.model.design()
        
        # Clear output.
        self.output.clear_output()

        # Build loads output.
        load_cols = ['x', 'y', 'z', 'x\'', 'y\'', 'z\'', 'Vx', 'Vy', 'Pz', 'Mx', 'My', 'Mz', 
                     'Vx\'', 'Vy\'', 'Pz\'', 'Mx\'', 'My\'', 'Mz\'']
        load_data = {}
        
        for k, v in self.model.loads.items():
            load_data[v.name] = np.concatenate((v.coords, v.coords_prime, v.load, v.load_prime))
            
        loads_df = pd.DataFrame.from_dict(load_data, orient='index', columns=load_cols)
        # Also include total resultant load.

        # Build total load.
        total_load_cols = ['Vx', 'Vy', 'Pz', 'Mx', 'My', 'Mz']
        total_load_data = {}
        total_load_data[v.name] = self.model.load
        total_load_df = pd.DataFrame.from_dict(total_load_data, orient='index', columns=total_load_cols)

        # Build weld segment output
        weld_cols = ['xi [in]', 'yi [in]', 'xj [in]', 'yj [in]', 'L [in]', 'xc [in]', 'yc [in]', 
                     'Ixx [in^4]', 'Iyy [in^4]', 'Lcx [in^3]', 'Lcy [in^3]',
                     'xi\' [in]', 'yi\' [in]', 'xj\' [in]', 'yj\' [in]', 'xc\' [in]', 'yc\' [in]', 
                     'Ixx\' [in^4]', 'Iyy\' [in^4]', 'Ixy\' [in^4]']
        stress_cols = ['sxi [ksi]', 'syi [ksi]', 'szi [ksi]', 
                       'sxj [ksi]', 'syj [ksi]', 'szj [ksi]', 
                       'si [ksi]', 'sj [ksi]']
        weld_data = {}
        stress_data = {}
        
        for k, v in self.model.welds.items():
            weld_data[v.name] =  np.concatenate((v.i[:2], v.j[:2], 
                                           np.array([v.L]), 
                                           v.cg[:2], 
                                           np.array([v.I[1], v.I[0]]), 
                                           v.Lc[:2],
                                           v.i_prime[:2], v.j_prime[:2],
                                           v.d_prime[:2], 
                                           np.array([v.I_prime[1], v.I_prime[0], v.Ixy_prime])))
            stress_data[v.name] = np.concatenate((v.si, v.sj, np.array([v.si_mag, v.sj_mag])))
            
        welds_df = pd.DataFrame.from_dict(weld_data, orient='index', columns=weld_cols)
        stress_df = pd.DataFrame.from_dict(stress_data, orient='index', columns=stress_cols)

        
        # Build weld group output
        group_cols = ['A', 'xc', 'yc', 'Ixx', 'Iyy', 'Ixy', 'Ip'] 
        group_data = {}
        g = self.model
        group_data[g.name] =  np.array([g.A, g.cg[0], g.cg[1], g.I[1], g.I[0], g.Ixy, g.Ip])
        group_df = pd.DataFrame.from_dict(group_data, orient='index', columns=group_cols)

        # Build design output
        design_cols = ['Fexx', 'phi', 'FW', 'Rn Req.', 'Throat Req.', 'Fillet Size', 'phi*Rn', 'Rn'] 
        design_data = {}
        design_data[g.name] =  np.array([g.Fexx, g.phi, g.FW, g.Rn_req, g.throat_req, g.fillet, g.phi_Rn, g.Rn])
        design_df = pd.DataFrame.from_dict(design_data, orient='index', columns=design_cols)
        
        # Plot the welds.
        # lines = [[(xi, yi), (xj, yj)], [(), ()], ...)
        lines = []
        for k, v in self.model.welds.items():
            lines.append([(v.i[0], v.i[1]), (v.j[0], v.j[1])])
        lc = LineCollection(lines)
        fig = plt.figure()
        ax1 = fig.add_subplot(1, 1, 1)
        ax1.add_collection(lc)
        ax1.axis('equal')
        x = [i[0] for j in lines for i in j]
        y = [i[1] for j in lines for i in j]
        ax1.scatter(x, y)
        
        # Show output.
        with self.output:
            plt.show()
            
            display(wg.HTMLMath("<h3> Weld Segment Properties </h3>"))
            display(welds_df)

            display(wg.HTMLMath("<h3> Weld Segment Stresses </h3>"))
            display(stress_df)
            
            display(wg.HTMLMath("<h3> Loads </h3>"))
            display(loads_df)

            display(wg.HTMLMath("<h3> Total Load </h3>"))
            display(total_load_df)

            display(wg.HTMLMath("<h3> Weld Group Properties </h3>"))
            display(group_df)

            display(wg.HTMLMath("<h3> Weld Design </h3>"))
            display(design_df)

In [3]:
# Create the calc.
group = Weld_Group([Weld(name='1', xj=6.0)], 
                   [Weld_Load(name='1', pz=100, x=3.0)])

# Create the layout.
layout = Weld_Layout(group)

## Discussion:

The Steel Construction Manual describes two methods for determining the capacity of weld groups: the (1) Instantaneous Center of Rotation Method (ICRM), and the (2) Elastic Method (EM).

### Instantaneous Center of Rotation Method
This method is able to ultilize an ultimate strength approach by accounting for the varying load angle and a non-linear load-deformation relationship in discretized segments of each weld line. The resisting force in each discrete weld segment is perpendicular to a radius constructed from an instantaneous center of rotation to the the centroid of the segment. This approach is computationally taxing and is best suited to computerized solution methods. Part 8 of the Manual provides pre-computed tables of strength coefficients for various common combinations of welds.

This method is not addressed by this program.

### Elastic Method
This method resolves loads into direct shear and shear resultants from eccentric forces and moments. This method of analysis gives conservative results, sometimes very conservative, compared to the ICRM and does not produce a consistent factor of safety for different configurations. The methods of analysis employed in this program accommodates unsymmetric bending with loading about the set of axes used to define the coordinates of the weld group. Note that the elastic method of analysis presented in some widely circulated spreadsheets does not fully support unsymmetric bending with loading defined about an arbitrary axis. Rather, the angle between the principal axes and the axes used to define the cross-section must be 0. This is not usually a convenient way to define loading on sections with doubly non-symmetric shapes.

## References:

1. Steel Construction Manual, AISC
2. Steel Design, 4th Ed., Segui
3. Design of Welded Structures, Blodgett
4. Boresi - Advanced Mechanics of Materials, 5th Ed - 1993

## Change Log:
- Created: Jan. 4, 2021 by SJ
- Released:

## Comments:
- Possibly incorporate some base metal checks.
- Show the CG, center of load, neutral axis (see Boresi).
- Allow the user to define weld segments arbitrarily or to pick from a list of predefined shapes.